In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import datetime
import matplotlib
import copy
import numpy.matlib as npm
from sklearn import linear_model
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import os
import random
#import lightgbm as lgb
import seaborn as sns
from statsmodels.tsa.arima_model import ARIMA
!pip install pmdarima
import pmdarima as pm
#from research_functions import *
import warnings
warnings.filterwarnings("ignore")
import textwrap


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.3 MB/s eta 0:00:00


In [ ]:

path = "/content/"
#path_output = '/data/market_maker/p&l_png'
path_output = '/content/output'
path_alpha = '/data/market_maker/alpha'
path_feat = '/data/market_maker/feature_test/results/'
path_feat_a = '/data/market_maker/feature_test_a/results/'



In [ ]:

def op_simulate(expr, freq='3D', neut=0, alloc_usdt=10000000, slippage=0,
                spread_slippage_pct=0, universe='top200', pos_limit=0.01,
                sd='2020-01-01', ed ='2023-04-30', live=False, save_pos=False,
                save_alpha=False, save_alpha_tmp=False, mktneut=True):
    univese_list = []
    print('live: ', live)
    if freq == '30M':
        factor = 24 * 2
        s = 50
        l = 150
    elif freq == '5M':
        factor = 24 * 12
        s = 50 * 6
        l = 150 * 6
    elif freq == '1M':
        factor = 24 * 60
        s = 50 * 30
        l = 150 * 30
    elif freq == '2H':
        factor = 12
        s = 10
        l = 40
    elif freq == '4H':
        factor = 6
        s = 5
        l = 20
    elif freq == '8H':
        factor = 3
        s = 3
        l = 14
    elif freq == '12H':
        factor = 2
        s = 2
        l = 8
    elif freq == '1D':
        factor = 1
        s = 1
        l = 4
    elif freq == '3D':
        factor = 1/3
        s = 1
        l = 4
    else:
        print('wrong frequency')
    '''
    if live:
        path_data = "/content/data_sampled/" + freq + "/All"
    else:
        path_data = "/content/data_sampled/" + freq + "/All"
    '''
    if universe == 'top10':
        universe_list = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'XRPUSDT', 'ADAUSDT',
                         'SOLUSDT', 'TRXUSDT', 'LTCUSDT', 'MATICUSDT', 'DOTUSDT']
    elif universe == 'top20':
        universe_list = ['BTCUSDT',	'ETHUSDT',	'BNBUSDT',	'XRPUSDT',	'ADAUSDT',
                         'DOGEUSDT',	'SOLUSDT',	'MATICUSDT',	'TRXUSDT', 'LTCUSDT',
                         'DOTUSDT',	'AVAXUSDT',	'ATOMUSDT',	'UNIUSDT', 'XMRUSDT',
                         'ETCUSDT',	'XLMUSDT',	'BCHUSDT',	'FILUSDT',	'HBARUSDT',
                         'LINKUSDT']
    elif universe == 'top20new':
        universe_list = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'XRPUSDT', 'ADAUSDT',
                         'SOLUSDT', 'MATICUSDT', 'TRXUSDT', 'LTCUSDT', 'DOTUSDT',
                         'AVAXUSDT', 'ATOMUSDT', 'UNIUSDT', 'ETCUSDT', 'XLMUSDT',
                         'BCHUSDT', 'FILUSDT', 'HBARUSDT', 'LINKUSDT', 'APTUSDT']
    elif universe == 'top40':
        universe_list = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'XRPUSDT', 'ADAUSDT',
                         'DOGEUSDT', 'SOLUSDT', 'MATICUSDT', 'TRXUSDT', 'LTCUSDT',
                         'DOTUSDT', 'AVAXUSDT', 'ATOMUSDT', 'UNIUSDT', 'XMRUSDT',
                         'ETCUSDT', 'XLMUSDT', 'BCHUSDT', 'FILUSDT', 'HBARUSDT',
                         'VETUSDT', 'NEARUSDT', 'GRTUSDT', 'ALGOUSDT', 'SANDUSDT',
                         'EOSUSDT', 'EGLDUSDT', 'AAVEUSDT', 'FTMUSDT', 'MANAUSDT',
                         'XTZUSDT', 'THETAUSDT','AXSUSDT','NEOUSDT', 'CHZUSDT',
                         'CRVUSDT', 'MKRUSDT', 'SNXUSDT' ]
    elif universe == 'top50':
        universe_list = ['BTCUSDT',	'ETHUSDT',	'BNBUSDT',	'XRPUSDT',	'ADAUSDT',
                         'DOGEUSDT',	'SOLUSDT',	'MATICUSDT',	'TRXUSDT',	'LTCUSDT',
                         'DOTUSDT',	'AVAXUSDT',	'ATOMUSDT',	'UNIUSDT',	'XMRUSDT',
                         'ETCUSDT',	'XLMUSDT',	'BCHUSDT',	'ICPUSDT',	'LDOUSDT',
                         'FILUSDT',	'APTUSDT',	'HBARUSDT',	'VETUSDT',  'ARBUSDT',
                         'NEARUSDT',	'QNTUSDT',	'GRTUSDT',	'APEUSDT',	'ALGOUSDT',
                         'SANDUSDT',	'EOSUSDT',	'EGLDUSDT',	'AAVEUSDT',	'RNDRUSDT',
                         'OPUSDT',	'FTMUSDT',	'MANAUSDT',	'XTZUSDT',	'THETAUSDT',
                         'STXUSDT',	'CFXUSDT',	'AXSUSDT',	'FLOWUSDT',	'NEOUSDT',
                         'CHZUSDT',	'CRVUSDT',	'IMXUSDT',	'MKRUSDT',	'SNXUSDT',
                         'INJUSDT', 'LINKUSDT']

    elif universe == 'top70':
        universe_list = [ 'XRPUSDT',  'ADAUSDT',  'DOGEUSDT',  'MATICUSDT', 'TRXUSDT',
                         'AVAXUSDT',  'XLMUSDT',  'LDOUSDT',   'HBARUSDT', 'ARBUSDT',
                         'NEARUSDT',  'GRTUSDT',  'APEUSDT',  'ALGOUSDT', 'SANDUSDT',
                         'EOSUSDT', 'RNDRUSDT', 'OPUSDT',   'FTMUSDT',  'MANAUSDT',
                         'XTZUSDT',  'THETAUSDT', 'STXUSDT',  'CFXUSDT',   'FLOWUSDT',
                         'CHZUSDT',  'CRVUSDT',  'IMXUSDT',  'SNXUSDT',  'INJUSDT',
                         'KLAYUSDT', 'KAVAUSDT', 'IOTAUSDT', 'WOOUSDT',  'RUNEUSDT',
                         'LRCUSDT',  'ZILUSDT',  'MASKUSDT', 'ENJUSDT',  'CVXUSDT',
                         'DYDXUSDT', 'BATUSDT',  '1INCHUSDT',  'ROSEUSDT', 'XEMUSDT',
                         'ENSUSDT',  'QTUMUSDT', 'HOTUSDT',  'RVNUSDT',  'BALUSDT',
                         'ANKRUSDT', 'CELOUSDT', 'AUDIOUSDT',    'SXPUSDT',  'OCEANUSDT',
                         'ICXUSDT',  'GMTUSDT',  'ARUSDT',   'IOTXUSDT', 'JASMYUSDT',
                         'GALAUSDT', 'ONTUSDT',  'SUSHIUSDT', 'WAVESUSDT',    'ZRXUSDT',
                         'ONEUSDT',  'BANDUSDT', 'IOSTUSDT', 'SFPUSDT',  'CELRUSDT',
                         'SKLUSDT',  'TOMOUSDT', 'RSRUSDT', 'LINKUSDT']
    elif universe == 'top100':
        universe_list = ['BTCUSDT',	'ETHUSDT',	'BNBUSDT',	'XRPUSDT',	'ADAUSDT',
                         'DOGEUSDT',	'SOLUSDT',	'MATICUSDT',	'TRXUSDT',	'LTCUSDT',
                         'DOTUSDT',	'AVAXUSDT',	'ATOMUSDT',	'UNIUSDT',	'XMRUSDT',
                         'ETCUSDT',	'XLMUSDT',	'BCHUSDT',	'ICPUSDT',	'LDOUSDT',
                         'FILUSDT',	'APTUSDT',	'HBARUSDT',	'ARBUSDT',	'NEARUSDT',
                         'QNTUSDT',	'GRTUSDT',	'APEUSDT',	'ALGOUSDT',	'SANDUSDT',
                         'EOSUSDT',	'EGLDUSDT',	'AAVEUSDT',	'RNDRUSDT',	'OPUSDT',
                         'FTMUSDT',	'MANAUSDT',	'XTZUSDT',	'THETAUSDT',	'STXUSDT',
                         'CFXUSDT',	'AXSUSDT',	'FLOWUSDT',	'NEOUSDT',	'CHZUSDT',
                         'CRVUSDT',	'IMXUSDT',	'MKRUSDT',	'SNXUSDT',	'INJUSDT',
                         'KLAYUSDT',	'KAVAUSDT',	'IOTAUSDT',	'ZECUSDT',	'DASHUSDT',
                         'WOOUSDT',	'RUNEUSDT',	'LRCUSDT',	'ZILUSDT',	'MASKUSDT',
                         'ENJUSDT',	'CVXUSDT',	'DYDXUSDT',	'BATUSDT',	'1INCHUSDT',
                         'ROSEUSDT',	'XEMUSDT',	'ENSUSDT',	'QTUMUSDT',	'HOTUSDT',
                         'COMPUSDT',	'RVNUSDT',	'BALUSDT',	'ANKRUSDT',	'CELOUSDT',
                         'AUDIOUSDT',	'SXPUSDT',	'OCEANUSDT',	'ICXUSDT',	'GMTUSDT',
                         'KSMUSDT',	'ARUSDT',	'IOTXUSDT',	'JASMYUSDT',	'YFIUSDT',
                         'GALAUSDT',	'ONTUSDT',	'SUSHIUSDT',	'WAVESUSDT',	'ZRXUSDT',
                         'ONEUSDT',	'BANDUSDT',	'IOSTUSDT',	'SFPUSDT',	'CELRUSDT',
                         'SKLUSDT',	'TOMOUSDT',	'RSRUSDT', 'LINKUSDT']
    elif universe == 'top200':
        universe_list = ['CVCUSDT', 'ASTRUSDT', 'XTZUSDT', 'EDUUSDT', 'BELUSDT',
                         'OGNUSDT', 'DUSKUSDT', 'ETCBUSD', 'ARUSDT', 'GMTUSDT',
                         'GMXUSDT', 'SUSHIUSDT', 'RUNEUSDT', 'KSMUSDT', 'PERPUSDT',
                         'EOSUSDT', 'BNBUSDT', 'GALABUSD', 'ATOMUSDT', 'DOGEBUSD',
                         'UNIUSDT', 'LDOBUSD', 'SOLUSDT', 'DASHUSDT', 'HBARUSDT',
                         'ZENUSDT', 'AGIXBUSD', 'TRBUSDT', 'LEVERUSDT', 'COTIUSDT',
                         'ICPUSDT', 'ALGOUSDT', 'PEOPLEUSDT', 'ICXUSDT', 'ZILUSDT',
                         'FETUSDT', 'RAYUSDT', 'XMRUSDT', 'XRPBUSD', 'QTUMUSDT',
                         'FTTUSDT', 'LINKBUSD', 'ADAUSDT', 'DARUSDT', 'DODOBUSD',
                         'ENJUSDT', 'TOMOUSDT', 'DOTBUSD', 'TLMBUSD', 'FLOWUSDT',
                         'KNCUSDT', 'BTCBUSD', 'BANDUSDT', 'APEBUSD', 'ETHUSDT',
                         'SANDUSDT', 'CVXBUSD', 'AVAXUSDT', 'FLMUSDT', 'ROSEUSDT',
                         'STGUSDT', 'ENSUSDT', 'MATICBUSD', 'FTMUSDT', 'SNXUSDT',
                         'RADUSDT', 'HOOKUSDT', 'INJUSDT', 'TRXBUSD', 'FILBUSD',
                         'LTCBUSD', 'RLCUSDT', 'AXSUSDT', 'ALPHAUSDT', 'APTBUSD',
                         'ALICEUSDT', 'QNTUSDT', 'RENUSDT', 'GRTUSDT', 'MAGICUSDT',
                         'DGBUSDT', 'COMPUSDT', 'YFIUSDT', 'JASMYUSDT', 'AMBBUSD',
                         'OCEANUSDT', 'SPELLUSDT', 'NEARBUSD', 'WAVESUSDT', 'STORJUSDT',
                         'ACHUSDT', 'SXPUSDT', 'CHRUSDT', 'SFPUSDT', 'CHZUSDT',
                         'GALBUSD', 'KLAYUSDT', 'FXSUSDT', 'STMXUSDT', 'IMXUSDT',
                         'RSRUSDT', 'ARPAUSDT', 'MTLUSDT', 'LINAUSDT', 'ZECUSDT',
                         'LQTYUSDT', 'BCHUSDT', 'PHBBUSD', 'ATAUSDT', 'KAVAUSDT',
                         'SSVUSDT', 'GTCUSDT', 'LDOUSDT', 'UNIBUSD', 'LITUSDT',
                         'SOLBUSD', 'LEVERBUSD', 'IOTXUSDT', 'THETAUSDT', 'AGIXUSDT',
                         'SRMUSDT', 'GALAUSDT', 'HNTUSDT', 'DOGEUSDT', 'BNBBUSD',
                         'AAVEUSDT', 'GMTBUSD', 'WOOUSDT', 'ETCUSDT', 'OPUSDT',
                         'CKBUSDT', 'EGLDUSDT', 'MINAUSDT', 'SCUSDT', 'STXUSDT',
                         'CELRUSDT', 'CTKUSDT', 'SUIUSDT', 'SANDBUSD', 'BTSUSDT',
                         'ETHBUSD', 'APEUSDT', 'HFTUSDT', 'XEMUSDT', 'ZRXUSDT',
                         'JOEUSDT', 'AVAXBUSD', 'CVXUSDT', 'API3USDT', 'IOSTUSDT',
                         'LPTUSDT', 'ANKRUSDT', 'ARBUSDT', 'LRCUSDT', 'BTCUSDT',
                         'ONTUSDT', 'XRPUSDT', 'FTTBUSD', 'VETUSDT', 'COCOSUSDT',
                         'DOTUSDT', 'TLMUSDT', 'CRVUSDT', 'LINKUSDT', 'ADABUSD',
                         'BLZUSDT', 'AUCTIONBUSD', 'DYDXUSDT', 'TRUUSDT', 'DENTUSDT',
                         'XVSUSDT', 'NEOUSDT', 'SKLUSDT', 'AUDIOUSDT', 'ANTUSDT',
                         'APTUSDT', 'IDEXUSDT', 'BATUSDT', 'FILUSDT', 'LTCUSDT',
                         'BNXUSDT', 'BALUSDT', 'MANAUSDT', 'ONEUSDT', 'MATICUSDT',
                         '1INCHUSDT', 'UNFIUSDT', 'TRXUSDT', 'XLMUSDT', 'FTMBUSD',
                         'HOTUSDT', 'BAKEUSDT', 'MASKUSDT', 'REEFUSDT', 'OMGUSDT',
                         'MKRUSDT', 'IDUSDT', 'PHBUSDT', 'UMAUSDT', 'TUSDT',
                         'IOTAUSDT', 'RNDRUSDT', 'ANCBUSD', 'CELOUSDT', 'RVNUSDT',
                         'HIGHUSDT', 'NKNUSDT', 'RDNTUSDT', 'CFXUSDT', 'GALUSDT',
                         'WAVESBUSD', 'NEARUSDT', 'ETHBTC', 'AMBUSDT', 'C98USDT',
                         'CTSIUSDT', 'LINKUSDT']
    elif universe == 'top180':
        universe_list = ['ASTRUSDT', 'XTZUSDT', 'EDUUSDT', 'BELUSDT', 'OGNUSDT',
                         'DUSKUSDT', 'ARUSDT', 'GMTUSDT', 'GMXUSDT', 'SUSHIUSDT',
                         'RUNEUSDT', 'KSMUSDT', 'PERPUSDT', 'EOSUSDT', 'BNBUSDT',
                         'ATOMUSDT', 'UNIUSDT', 'SOLUSDT', 'DASHUSDT', 'HBARUSDT',
                         'ZENUSDT', 'TRBUSDT', 'LEVERUSDT', 'COTIUSDT', 'ICPUSDT',
                         'ALGOUSDT', 'PEOPLEUSDT', 'ICXUSDT', 'ZILUSDT', 'FETUSDT',
                         'XMRUSDT', 'QTUMUSDT', 'ADAUSDT', 'DARUSDT', 'ENJUSDT',
                         'TOMOUSDT', 'FLOWUSDT', 'KNCUSDT', 'BANDUSDT', 'ETHUSDT',
                         'SANDUSDT', 'AVAXUSDT', 'FLMUSDT', 'ROSEUSDT', 'STGUSDT',
                         'ENSUSDT', 'FTMUSDT', 'SNXUSDT', 'RADUSDT', 'HOOKUSDT',
                         'INJUSDT', 'RLCUSDT', 'AXSUSDT', 'ALPHAUSDT', 'ALICEUSDT',
                         'QNTUSDT', 'RENUSDT', 'GRTUSDT', 'MAGICUSDT', 'DGBUSDT',
                         'COMPUSDT', 'YFIUSDT', 'JASMYUSDT', 'OCEANUSDT', 'SPELLUSDT',
                         'WAVESUSDT', 'STORJUSDT', 'ACHUSDT', 'SXPUSDT', 'CHRUSDT',
                         'SFPUSDT', 'CHZUSDT', 'KLAYUSDT', 'FXSUSDT', 'STMXUSDT',
                         'IMXUSDT', 'RSRUSDT', 'ARPAUSDT', 'MTLUSDT', 'LINAUSDT',
                         'ZECUSDT', 'LQTYUSDT', 'BCHUSDT', 'ATAUSDT', 'KAVAUSDT',
                         'SSVUSDT', 'GTCUSDT', 'LDOUSDT', 'LITUSDT', 'IOTXUSDT',
                         'THETAUSDT', 'AGIXUSDT', 'GALAUSDT', 'DOGEUSDT', 'AAVEUSDT',
                         'WOOUSDT', 'ETCUSDT', 'OPUSDT', 'CKBUSDT', 'EGLDUSDT',
                         'MINAUSDT', 'STXUSDT', 'CELRUSDT', 'CTKUSDT', 'SUIUSDT',
                         'APEUSDT', 'HFTUSDT', 'XEMUSDT', 'ZRXUSDT', 'JOEUSDT',
                         'CVXUSDT', 'API3USDT', 'IOSTUSDT', 'LPTUSDT', 'ANKRUSDT',
                         'ARBUSDT', 'LRCUSDT', 'BTCUSDT', 'ONTUSDT', 'XRPUSDT',
                         'VETUSDT', 'DOTUSDT', 'TLMUSDT', 'CRVUSDT', 'LINKUSDT',
                         'BLZUSDT', 'DYDXUSDT', 'TRUUSDT', 'DENTUSDT', 'XVSUSDT',
                         'NEOUSDT', 'SKLUSDT', 'AUDIOUSDT', 'ANTUSDT', 'APTUSDT',
                         'IDEXUSDT', 'BATUSDT', 'FILUSDT', 'LTCUSDT', 'BNXUSDT',
                         'BALUSDT', 'MANAUSDT', 'ONEUSDT', 'MATICUSDT', '1INCHUSDT',
                         'UNFIUSDT', 'TRXUSDT', 'XLMUSDT', 'HOTUSDT', 'BAKEUSDT',
                         'MASKUSDT', 'REEFUSDT', 'OMGUSDT', 'MKRUSDT', 'IDUSDT',
                         'PHBUSDT', 'UMAUSDT', 'TUSDT', 'IOTAUSDT', 'RNDRUSDT',
                         'CELOUSDT', 'RVNUSDT', 'HIGHUSDT', 'NKNUSDT', 'RDNTUSDT',
                         'CFXUSDT', 'GALUSDT', 'NEARUSDT', 'AMBUSDT', 'C98USDT',
                         'CTSIUSDT','LINKUSDT']
    else:
        print("univese not available")
    var_list2 = ['PERP_Close', 'PERP_High', 'PERP_Low', 'PERP_Numtrades', 'PERP_Open',
                 'PERP_Takerbuyvol', 'PERP_Volume', 'PERP_VWAP', 'PERP_VWAP_from4h', 'PERP_VWAP_from30min',
                 'PERP_Volume_qa', 'PERP_Takerbuyvol_qa'] #PERP_Spread
    '''
    import pandas as pd
    url='https://drive.google.com/file/d/0B6GhBwm5vaB2ekdlZW5WZnppb28/view?usp=sharing'
    url='https://drive.google.com/uc?id=' + url.split('/')[-2]
    '''
#     var_list = ['https://drive.google.com/file/d/18FEDou9s5P89bgn56GLTv87YdvW6AU1N/view?usp=drive_link',
#                 'https://drive.google.com/file/d/1XDPJOcuMyCpYuo881z0maALVAjl9OpF7/view?usp=drive_link',
#                 'https://drive.google.com/file/d/1rTimJW4tJDNLpiumS-PJ9K_p_DbK4tgp/view?usp=drive_link',
#                 'https://drive.google.com/file/d/1h-Nyw8nS8SDmK9g6TN_g21LEAUyVTzkD/view?usp=drive_link',
#                 'https://drive.google.com/file/d/1zkSL-QLIR7h5RgSOTT_bv-TDYoHgyzEE/view?usp=drive_link',
#                 'https://drive.google.com/file/d/1vtXA2_JL9H9Os8psF8EdaC_OvgKLzcKt/view?usp=drive_link',
#                 'https://drive.google.com/file/d/16D8u5sBEgUKCOWWZlPov-eEB7XSB4y-S/view?usp=drive_link',
#                 'https://drive.google.com/file/d/1At-metGZfuAwww9rVtx8ge7Upj3CP2JT/view?usp=drive_link',
#                 #'https://drive.google.com/file/d/1aNutsnfGwzfsYBp81n1MRGA4U-2KCMEN/view?usp=drive_link'
#                 ]

    var_list = ['MD_3D_PERP_Close.csv',
                'MD_3D_PERP_High.csv',
                'MD_3D_PERP_Low.csv',
                'MD_3D_PERP_Numtrades.csv',
                'MD_3D_PERP_Open.csv',
                'MD_3D_PERP_Takerbuyvol.csv',
                'MD_3D_PERP_Volume.csv',
                'MD_3D_PERP_VWAP.csv',
                #'DS_Store'
                ]

    if 'spot' in expr:
        var_list += [ 'SPOT_Close', 'SPOT_High', 'SPOT_Low', 'SPOT_Numtrades', 'SPOT_Open',
                     'SPOT_Takerbuyvol', 'SPOT_Volume']
    if 'okx' in expr:
        var_list += ['PERP_Close_okx', 'PERP_High_okx', 'PERP_Low_okx', 'PERP_Open_okx', 'PERP_Volume_okx']

    for i in range(len(var_list)):
#        var_path ='https://drive.google.com/uc?id=' + var_list[i].split('/')[-2]
         var_path = 'Data/' + var_list[i]
         print('this os the variable path', var_path)
         #print('pd.read_csv(' + var_path + ', index_col=[0], encoding = "utf-8")[["' + '", "'.join(universe_list) + '"]].loc["' + sd + '" : "' + ed + '"]', globals())
         exec('pd.read_csv("' + var_path + '", index_col=[0], encoding = "utf-8")[["' + '", "'.join(universe_list) + '"]].loc["' + sd + '" : "' + ed + '"]', globals())
         print('this is the try,',var_list2[i] + '= pd.read_csv("' + var_path + '"')

    try:
        spread= pd.read_csv("/data/market_datas/data_sampled/"+freq+"/All/MD_"+freq+"_PERP_Spread.csv", index_col=[0])[universe_list].loc[sd : ed]
    except Exception as e:
        print('error', e)
        pass
    print('simulating: ', expr, ' ', universe)

    filename_arr = ['vwap', 'volume', 'takerbuyvol', 'open', 'numtrades', 'low', 'high', 'close' ]
    path_arr = []
    for i in filename_arr:
        path_arr.append('Data/MD_3D_PERP_' + i.capitalize() + '.csv')
    mydict = dict(zip(filename_arr, path_arr))
    my_url = mydict.get(expr)
    print(my_url)
    #my_path = 'https://drive.google.com/uc?export=download&id='+ my_url.split('/')[-2]
    my_path=my_url
    print('My path is : ' + my_path)
    print('The Alpha expression is : ' + expr + " = pd.read_csv('" + my_path + "')")


    #alpha = exec(expr + " = pd.read_csv('" + (my_path) + "')", globals())
    #global alpha
    exec(expr + " = pd.read_csv('" + (my_path) + "')", globals())
    alpha = expr
    #exec(f"alpha = expr", globals())
    #print('Alpha: ' + alpha)

    ## Attempted fix, put close and other variables into a dictionary
#     var_dict = {}
#     var_dict[expr] = exec(expr + " = pd.read_csv('" + (my_path) + "')")
#     print(var_dict[expr])



    if save_alpha:
        path_alpha = '/content/alpha'
        if not os.path.exists(path_alpha):
            os.makedirs(path_alpha)
        alpha.to_csv(path_alpha + '/alpha_raw_step1.csv')
    #ret = close.pct_change()

    close.drop('Timestamp', axis = 'columns', inplace=True)
    #close.fillna(-1, inplace = True) ## Unsure if necessary
    ret = close/close.shift(1) - 1
    alpha = close
    if neut:
        s = 50
        l = 150
        mon2 = np.sign(ts_ma_crossover(close, s, l))
        alpha = cs_reg_residue(mon2, alpha)
    #print("alpha_raw:", '\n', alpha)
    if not mktneut:
        alpha = cs_booksize(alpha, alloc_usdt)
    else:
        alpha = cs_booksize(cs_mktneut(alpha), alloc_usdt)
    if save_alpha:
        if not os.path.exists(path_alpha):
            os.makedirs(path_alpha)
        alpha.to_csv(path_alpha + '/alpha_neut_booksize_step2.csv')
    if not mktneut:
        print("alpha = cs_booksize(alpha, alloc_usdt)", '\n', alpha)
    else:
        print("alpha = cs_booksize(cs_mktneut(alpha), alloc_usdt)", '\n', alpha)
    alpha = cs_expand_balance(cs_poslimit(alpha, pos_limit * alloc_usdt), alloc_usdt)
    print("cs_expand_balance(cs_poslimit(alpha, pos_limit * alloc_usdt), alloc_usdt)", '\n', alpha)
    if save_alpha:
        print('alpha position:', '\n', alpha.round(0))
        if not os.path.exists(path_alpha):
            os.makedirs(path_alpha)
        alpha.to_csv(path_alpha + '/alpha_poslimit_step3.csv')
    if spread_slippage_pct != 0:
        spread = spread.fillna(method='ffill').fillna(method='bfill')
        print("spread.shape", spread.shape)
        print("close.shape", close.shape)
        coinpnl = ts_delay(alpha, 1) * ret - alpha.diff().abs() * spread_slippage_pct * spread / close
    else:
        coinpnl = ts_delay(alpha, 1) * ret - alpha.diff().abs() * slippage

    numcoins = coinpnl.notnull().sum(axis=1)
    dailytvr = alpha.diff().abs().sum(axis=1) / alpha.abs().shift(1).sum(axis=1)
    dailytvr.replace(np.inf, np.nan, inplace=True)
    dailytvr.index = pd.to_datetime(dailytvr.index)
    tvr = dailytvr.mean() * factor
    coincumpnl = coinpnl.cumsum()
    coincumpnl.index = pd.to_datetime(coincumpnl.index)
    dailypnl = coinpnl.sum(axis=1)
    at_nan2zero(dailypnl)
    dailypnl.index = pd.to_datetime(dailypnl.index)
    equity = alloc_usdt + dailypnl.cumsum()
    equity.index = pd.to_datetime(equity.index)
    sp = dailypnl.mean() / dailypnl.std() * (365 * factor) ** 0.5
    maxequity = equity.rolling(len(equity), min_periods=1).max()
    dd = ((maxequity - equity) / alloc_usdt).max()
    ret = (equity.iloc[-1] - alloc_usdt) / alloc_usdt / len(equity) * 365 * factor
    fitness = sp * (abs(ret) / max(tvr, 0.125))
    shortsize = alpha[alpha < 0].sum(axis=1)
    longsize = alpha[alpha > 0].sum(axis=1)
    if len(equity) < 365:
        shortnum = alpha[alpha < 0].count(axis=1).mean()#.values[0]

        longnum = alpha[alpha > 0].count(axis=1).mean()#.values[0]

    else:
        shortnum = alpha[alpha < 0].count(axis=1).rolling(365).mean().values[-1]
        longnum = alpha[alpha > 0].count(axis=1).rolling(365).mean().values[-1]

    plt.figure(figsize=(20, 14))
    label = 'sp: ' + str("{:.1f}".format(sp)) + ' fit: ' + str("{:.2f}".format(fitness)) + ' ret: ' + str("{:.2f}".format(ret)) + ' dd: ' + str("{:.2f}".format(dd))  + ' tvr: ' + str("{:.1f}".format(tvr)) +' l: ' + str("{:.0f}".format(shortnum)) +' s: ' + str("{:.0f}".format(longnum)) + '\nfreq: ' + freq + ' slp: ' + str("{:.4f}".format(slippage)) + ' pos_limit: ' + str("{:.2f}".format(pos_limit))
    print(label)
    wrapped_expr = '\n'.join(textwrap.wrap('expression: ' + expr, width=100))
    plt.plot(equity, color='red', linewidth=1, label=label + '\n' + wrapped_expr)
    plt.grid(linestyle='--')
    plt.legend(loc='upper left')
    #plt.savefig(
        #path_output + "/{0:.2f}".format(sp) + "_slp" + str(slippage) + "_" + freq + "_" + universe + '_' + expr.replace(' ', '').replace('/', '').replace('"', '')[:50] + '.png')

    return equity, alpha


In [ ]:
op_simulate('close', freq='3D', neut=0, alloc_usdt=10000000, slippage=0,
            spread_slippage_pct=0, universe='top200', pos_limit=0.01, sd='2020-01-01', ed ='2023-04-30',
            live=False, save_pos=False, save_alpha=False, save_alpha_tmp=False, mktneut=True)

live:  False
this os the variable path Data/MD_3D_PERP_Close.csv


FileNotFoundError: [Errno 2] No such file or directory: 'Data/MD_3D_PERP_Close.csv'

In [ ]:
pip install gdown

ERROR: Operation cancelled by user


In [ ]:
pip install sourcedefender

     ━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.7/10.8 MB 3.3 MB/s eta 0:00:04
ERROR: Operation cancelled by user


In [ ]:
!python bb.py

python3: can't open file '/content/bb.py': [Errno 2] No such file or directory


In [ ]:
url='https://drive.google.com/file/d/1XDPJOcuMyCpYuo881z0maALVAjl9OpF7/view?usp=drive_link'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url)
print(df)

      Timestamp      BTCUSDT       ETHUSDT  BCHUSDT  XRPUSDT  EOSUSDT  \
0    2019-09-10    17920.664           NaN      NaN      NaN      NaN   
1    2019-09-13    35576.511           NaN      NaN      NaN      NaN   
2    2019-09-16    60445.600           NaN      NaN      NaN      NaN   
3    2019-09-19    63538.139           NaN      NaN      NaN      NaN   
4    2019-09-22    73137.051           NaN      NaN      NaN      NaN   
..          ...          ...           ...      ...      ...      ...   
466  2023-07-06   966022.074  6.988007e+06      NaN      NaN      NaN   
467  2023-07-09  1197803.366  7.514647e+06      NaN      NaN      NaN   
468  2023-07-12   890316.547  5.464168e+06      NaN      NaN      NaN   
469  2023-07-15  1659774.013  1.179057e+07      NaN      NaN      NaN   
470  2023-07-18   629541.841  4.933409e+06      NaN      NaN      NaN   

     LTCUSDT  TRXUSDT  ETCUSDT  LINKUSDT  ...  XVSUSDT  ETHBTC  BLURUSDT  \
0        NaN      NaN      NaN       NaN  ...  

In [ ]:
!unzip /content/data_sampled.zip

unzip:  cannot find or open /content/data_sampled.zip, /content/data_sampled.zip.zip or /content/data_sampled.zip.ZIP.
